# Week 02 - Deep Learning Repetition

## Data Augmentation

Even though the metrics looked quite good in the basic model, the performance of classifying custom hand-written digits was sub-optimal.

In this notebook, we address this issue by performing **data augmentation** on the dataset to account for variations such as rotations, different pencil shapes, blurs, and more.

These modifications should improve the model's ability to recognize custom hand-written digits. 📝

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from torchinfo import summary
import sklearn.metrics
from tqdm.notebook import tqdm
from pathlib import Path
import shutil
import onnxruntime

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device.upper()}")

## Load the MNIST Dataset 

### Data Augmentation

There are a few options to augment the image data:

- Randomly crop the image: RandomCrop(Output Image Size, Padding of the Image)

    `torchvision.transforms.RandomCrop(28, 4)`

- Randomly rotate the image: RandomRotation(Max Rotation in Degrees)

    `torchvision.transforms.RandomRotation(5)`

- Randomly blur the image: GaussianBlur(Kernel Size) -> larger kernel means more blur

    `torchvision.transforms.GaussianBlur(kernel_size = (5,5))`

- Randomly change the colors in the image: ColorJitter(Brightness, Contrast, Saturation, Hue)

    `torchvision.transforms.ColorJitter(brightness=(0.5,1.5), contrast=3, saturation=(0.3,1.5), hue=(-0.1,0.1))`


In [15]:
# Define the transformation
image_transformation = transforms.Compose(
    [
        # '''TODO: Insert data augmentation here.'''
        transforms.ToTensor(),
    ]
)

In [16]:
mnist_directory = Path.cwd() / ".mnist_data" / "2_augmented"

# Delete the existing MNIST data directory if it exists
if mnist_directory.exists() and mnist_directory.is_dir():
    shutil.rmtree(mnist_directory)

In [ ]:
# Load the MNIST dataset
dataset_mnist_train = datasets.MNIST(
    mnist_directory, train=True, download=True, transform=image_transformation
)
dataset_mnist_test = datasets.MNIST(
    mnist_directory, train=False, download=True, transform=image_transformation
)

Split the training dataset into training and validation sets using PyTorch's `random_split` function.

In [18]:
# Split the training dataset into training and validation sets
train_size = int(0.8 * len(dataset_mnist_train))
val_size = len(dataset_mnist_train) - train_size
dataset_mnist_train, dataset_mnist_val = random_split(
    dataset_mnist_train, [train_size, val_size]
)

The `DataLoader` class is useful to create batch sizes of the data and to shuffle it.

In [19]:
# Create DataLoaders
train_loader = DataLoader(dataset_mnist_train, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset_mnist_val, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset_mnist_test, batch_size=64, shuffle=True)

## Visualize the Dataset

Let us briefly look at the data 👀

In [ ]:
images, labels = next(iter(train_loader))

print(f"Image Batch shape: {images.shape}")
print(f"Label Batch shape: {labels.shape}")

In [ ]:
fig, axes = plt.subplots(3, 3)
fig.suptitle("MNIST Dataset Samples", fontsize=16)

for ax in axes.flat:
    idx = torch.randint(0, len(images), (1,))
    ax.imshow(images[idx].squeeze(), cmap="gray")
    ax.set_title(f"Label: {labels[idx].item()}")
    ax.axis("off")

plt.tight_layout()
plt.show()

## Model Architecture

We still use the same simple CNN to better compare the results (change one thing at a time).

In [22]:
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(in_features=4 * 14 * 14, out_features=10),
)

In [ ]:
print(summary(model=model, input_size=images.shape))
model_output = model(images)
print(f"Model output shape: {model_output.shape}")

## Model Training

In [ ]:
EPOCHS = 2
LEARNING_RATE = 0.001
BATCH_INTERVAL = 100  # Calculate losses after every 100 batches

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.CrossEntropyLoss()

train_loss_history = []
val_loss_history = []

model.to(device)

for epoch in range(EPOCHS):
    model.train()
    running_train_loss = 0.0
    for batch_idx, (image_batch, output_batch) in enumerate(
        tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{EPOCHS}"), start=1
    ):
        image_batch, output_batch = image_batch.to(device), output_batch.to(device)
        optimizer.zero_grad()
        model_output = model(image_batch)
        loss = loss_function(model_output, output_batch)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()

        if (batch_idx) % BATCH_INTERVAL == 0:
            avg_train_loss = running_train_loss / (batch_idx)
            train_loss_history.append(avg_train_loss)
            print(f"After batch {batch_idx}: Train Loss: {avg_train_loss:.4f}")

            # Evaluate validation loss
            model.eval()
            running_val_loss = 0.0
            with torch.no_grad():
                for val_image_batch, val_output_batch in tqdm(
                    val_loader, desc="Validation"
                ):
                    val_image_batch, val_output_batch = (
                        val_image_batch.to(device),
                        val_output_batch.to(device),
                    )
                    val_model_output = model(val_image_batch)
                    val_loss = loss_function(val_model_output, val_output_batch)
                    running_val_loss += val_loss.item()
            avg_val_loss = running_val_loss / len(val_loader)
            val_loss_history.append(avg_val_loss)
            print(f"After batch {batch_idx}: Validation Loss: {avg_val_loss:.4f}")
            model.train()  # Switch back to training mode

    avg_epoch_train_loss = running_train_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Train Loss: {avg_epoch_train_loss:.4f}")

    # Evaluate validation loss at the end of each epoch
    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for val_image_batch, val_output_batch in tqdm(val_loader, desc="Validation"):
            val_model_output = model(val_image_batch)
            val_loss = loss_function(val_model_output, val_output_batch)
            running_val_loss += val_loss.item()
    avg_epoch_val_loss = running_val_loss / len(val_loader)
    val_loss_history.append(avg_epoch_val_loss)
    print(f"Epoch {epoch+1} - Validation Loss: {avg_epoch_val_loss:.4f}")

plt.plot(train_loss_history, label="Train Loss")
plt.plot(val_loss_history, label="Validation Loss")
plt.title("Loss")
plt.legend()
plt.show()

## Test the Model

Test the freshly trained model with a randomly drawn image from the dataset.

In [3]:
def load_onnx_model(path_to_model: Path):
    if path_to_model is None:
        raise ValueError("Path must be provided if onnx is True")
    return onnxruntime.InferenceSession(
        str(path_to_model), providers=["CPUExecutionProvider"]
    )


def predict_digit(
    input_tensor: torch.Tensor,
    model: torch.nn.Module = None,
    onnx: bool = False,
    onnx_model_path: Path = None,
) -> tuple[int, float]:
    if onnx:
        model_session = load_onnx_model(onnx_model_path)
        onnx_predictions = model_session.run(None, {"image": input_tensor})
        predictions = torch.tensor(np.array(onnx_predictions))
    else:
        predictions = model(input_tensor)

    predicted_number = torch.argmax(predictions)
    probabilities = F.softmax(predictions.clone().detach(), dim=-1).numpy().squeeze()
    confidence = probabilities[predicted_number]
    return predicted_number.item(), confidence

In [ ]:
sampling_idx = torch.randint(0, len(images), (1,))
test_img = images[sampling_idx]
test_label = labels[sampling_idx]
plt.imshow(test_img.numpy().squeeze(), cmap="grey")
plt.show()
predicted_number, confidence = predict_digit(test_img, model)
print(f"Predicted : {predicted_number} - Confidence: {100*confidence:.2f}%")
print(f"True label: {test_label.item()}")

### Metrics

Does it look good?

Since this is just one image from the **training** dataset, we need to evaluate how the model performs on data it has never seen before. This is where the **test** dataset comes in. In the following cell, we calculate various metrics to evaluate the model's performance.

The metrics we will calculate are Accuracy, Recall, F1 score, and Support:

- **Accuracy**: The ratio of correctly predicted instances to the total instances. It gives an overall measure of how well the model is performing.
- **Recall**: The ratio of correctly predicted positive observations to all the observations in the actual class. It tells us how well the model can identify positive instances.
- **F1 Score**: The weighted average of Precision and Recall. It is useful when you need a balance between Precision and Recall.
- **Support**: The number of actual occurrences of the class in the dataset. It helps in understanding the distribution of the dataset.

When interpreting these metrics, keep in mind:
- **Accuracy** can be misleading if the dataset is imbalanced.
- **Recall** is crucial when the cost of false negatives is high.
- **F1 Score** is useful when you need to balance Precision and Recall.
- **Support** helps in understanding the context of the other metrics.


In [ ]:
model.to(device)
model.eval()

test_predicted_output = []
test_true_output = []

with torch.no_grad():
    for batch_idx, (image_batch, output_batch) in enumerate(
        tqdm(test_loader, desc="Evaluating the model on the Test set")
    ):
        image_batch, output_batch = image_batch.to(device), output_batch.to(device)
        model_output = model(image_batch)
        predicted_numbers = torch.argmax(model_output, dim=1)
        test_predicted_output.extend(predicted_numbers.cpu().numpy())
        test_true_output.extend(output_batch.cpu().numpy())

cm_abs = sklearn.metrics.confusion_matrix(
    test_true_output, test_predicted_output, labels=[i for i in range(10)]
)
cm_norm = sklearn.metrics.confusion_matrix(
    test_true_output,
    test_predicted_output,
    labels=[i for i in range(10)],
    normalize="true",
)
cm_round = np.around(cm_norm, 2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

disp_abs = sklearn.metrics.ConfusionMatrixDisplay(
    confusion_matrix=cm_abs, display_labels=[i for i in range(10)]
)
disp_norm = sklearn.metrics.ConfusionMatrixDisplay(
    confusion_matrix=cm_round, display_labels=[i for i in range(10)]
)

classification_report_test = sklearn.metrics.classification_report(
    test_true_output, test_predicted_output, labels=[i for i in range(10)]
)

disp_abs.plot(ax=ax1)
ax1.set_title("Absolute")
ax1.grid(False)

disp_norm.plot(ax=ax2)
ax2.set_title("Normalized")
ax2.grid(False)
plt.show()

print(classification_report_test)

With a simple CNN and augmented data, the metrics tend to be lower than in the other notebooks. You can add another epoch, which might improve the scores. Nonetheless, let us see how well the model performs on your hand-written digits.

## Model Export

In [ ]:
FILE_NAME = "./2_augmented_mnist_model.onnx"

MODEL_FOLDER = Path.cwd() / "models"
MODEL_PATH = MODEL_FOLDER / FILE_NAME

# Create the directory if it doesn't exist
if not MODEL_FOLDER.exists():
    MODEL_FOLDER.mkdir(parents=True, exist_ok=True)
    print(f"Directory '{MODEL_FOLDER}' created.")
else:
    print(f"Directory '{MODEL_FOLDER}' already exists.")

In [ ]:
# Save the model
model.eval()
image = images[0].unsqueeze(0)
torch.onnx.export(
    model, image, MODEL_PATH, input_names=["image"], output_names=["digit"]
)

print(f"Model exported in ONNX format to: {MODEL_PATH}")

## Real-world Data

Most tutorials would just finish with the above evaluation. After all, the metrics look quite good, the majority of the manually provided samples to the model are correctly classified; what more do you want?

What is mostly missing, but what is the most common problem in practice, is, that there are real-world cases with real-world data. The model is performing quite well, but what if you provide completely novel hand-written digits to it that is maybe not nicely centred, or is drawn with a different pencil width, or rotated, etc.

In the following section, we add this real-world scenario by letting you draw your own hand-written digits and test your trained models against this custom dataset.

### Single Digit

To create a single digit that can be run in the `Real-World Model Inference` below:
1. Open the drawing tool to create hand-written digits yourself by running the next cell
1. Select the model you just created
1. Draw a digit
1. Click on 'Classify' to see how well the model predicts
1. Close the tool (or try more digits)

#### Custom Digits Test Dataset

To create a suite of digits ranging from 0 to 9 that can be run in the `Real-World Digits Comparison` below:
1. Open the drawing tool to create hand-written digits yourself by running the next cell
1. Click on `Create Custom Test Set`
1. Draw the digit that is prompted
1. Click on `Save and Next`
1. Once done you can change any drawn number by selecting it from the dropdown

In [ ]:
!python src/model_test_drawing.py

### Real-World Model Inference

In [ ]:
assert (
    Path.cwd() / "handwriting" / "my_digit.npy"
).exists(), "Draw and classify a digit first."
image_input_tensor = np.load("handwriting/my_digit.npy")

plt.cla()
plt.imshow(image_input_tensor[0].squeeze(), cmap="grey")
plt.title("Input Tensor")
plt.show()

try:
    model.eval()
    pt_predicted_number, pt_confidence = predict_digit(
        torch.tensor(image_input_tensor), model
    )
    print(
        f"PyTorch model prediction: {pt_predicted_number} - Confidence: {100*pt_confidence:.2f}%"
    )
except NameError:
    print("No PyTorch model present, skipping...")

onnx_predicted_number, onnx_confidence = predict_digit(
    image_input_tensor, onnx=True, onnx_model_path=MODEL_PATH
)
print(
    f"ONNX model prediction:    {onnx_predicted_number} - Confidence: {100*onnx_confidence:.2f}%"
)

#### Real-World Digits Comparison

The plot lets you compare the performance of all the models from the following notebooks (they will have the same output, just with their model).

In [ ]:
assert (
    Path.cwd() / "handwriting" / "digit_0.npy"
).exists(), "Draw and classify a digit first."
directory_path = Path("handwriting/")

fig, axes = plt.subplots(10, 2, figsize=(5, 20))

for i in range(10):
    file_path = directory_path / f"digit_{i}.npy"
    image_input_tensor = np.load(file_path)

    axes[i, 0].imshow(image_input_tensor.squeeze(), cmap="gray")
    axes[i, 0].set_title(f"Digit {i}")
    axes[i, 0].axis("off")

    onnx_predicted_number, onnx_confidence = predict_digit(
        image_input_tensor, onnx=True, onnx_model_path=MODEL_PATH
    )

    axes[i, 1].text(
        0.5, 0.6, f"Model prediction: {onnx_predicted_number}", fontsize=12, ha="center"
    )
    axes[i, 1].text(
        0.5, 0.4, f"Confidence: {100*onnx_confidence:.2f}%", fontsize=12, ha="center"
    )
    axes[i, 1].axis("off")

# Adjust layout
plt.tight_layout()
plt.show()

It seems that even though we augmented the dataset with one or more transformations, the model is still not able to classify well on your drawings.

The model may still be wrong, but hopefully with a lower confidence than the basic model.

In the final notebook, we will address this issue!